[문제] Managing Conversation History  
- trim_messages()  
- RunnablePassthrough  
- itemgetter()  

세션ID 설정하여, 대화 진행합니다.(multi-turn conversation)  
LLM 모델이 과거대화를 알지(기억) 못하는 상황을 만드세요.  


In [ ]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.messages import trim_messages
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.chat_history import (
    BaseChatMessageHistory, 
    InMemoryChatMessageHistory
)


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 7. trim 설정
trimmer = trim_messages(
    max_tokens=65,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 8. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm 
)

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}},     
    )
    print('\n'+'*' * 100 +'\n')

죄송하지만 그 질문에 대한 답변은 제공할 수 없습니다. 이력서를 작성을 위해 다른 정보를 제공해 주시면 감사하겠습니다.
****************************************************************************************************



In [18]:
get_history('1234')

InMemoryChatMessageHistory(messages=[HumanMessage(content='이병헌 파이썬 경력없음', additional_kwargs={}, response_metadata={}), AIMessage(content='**이병헌**\n\n---\n\n**연락처**  \n이메일: leebyunghun@example.com  \n전화번호: 010-1234-5678  \n주소: 서울특별시\n\n---\n\n**목표**  \n파이썬을 통해 새로운 기술을 배우고 프로그래밍 분야에서 경력을 쌓아나가는 것을 목표로 하는 열정적인 신입 개발자입니다. 정확성과 창의성을 바탕으로 문제를 해결하고, 팀과의 원활한 협업을 통해 프로젝트의 성공에 기여하고자 합니다.\n\n---\n\n**교육**  \n- **서울대학교**  \n  컴퓨터공학과 미졸업\n\n---\n\n**기술 역량**  \n- **프', additional_kwargs={}, response_metadata={'finish_reason': 'length', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8'}, id='run-253e00bc-3a5f-4203-9c43-bc5b5d95f8ad-0'), HumanMessage(content='귀사의 꽃이 되고싶습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content="[이력서]\n\n**개인 정보**\n- 이름: 김지원\n- 이메일: jiwan.kim@email.com\n- 전화번호: 010-1234-5678\n- 주소: 서울특별시 강남구 테헤란로\n\n**목표**\n귀사의 발전과 함께 성장하며, 제 역량을 최대한 발휘하여 귀사의 주요 구성원, 즉 '꽃'이 되고자 합니다. \n\n**학력**\n- **서울대학교**  \n  경영학 학사  \n  2018년 3월 - 2022년 2월\n\n**경력**\

In [19]:
for info in get_history('1234').messages:
    print(info.content)
    print('*' * 50)

이병헌 파이썬 경력없음
**************************************************
**이병헌**

---

**연락처**  
이메일: leebyunghun@example.com  
전화번호: 010-1234-5678  
주소: 서울특별시

---

**목표**  
파이썬을 통해 새로운 기술을 배우고 프로그래밍 분야에서 경력을 쌓아나가는 것을 목표로 하는 열정적인 신입 개발자입니다. 정확성과 창의성을 바탕으로 문제를 해결하고, 팀과의 원활한 협업을 통해 프로젝트의 성공에 기여하고자 합니다.

---

**교육**  
- **서울대학교**  
  컴퓨터공학과 미졸업

---

**기술 역량**  
- **프
**************************************************
귀사의 꽃이 되고싶습니다.
**************************************************
[이력서]

**개인 정보**
- 이름: 김지원
- 이메일: jiwan.kim@email.com
- 전화번호: 010-1234-5678
- 주소: 서울특별시 강남구 테헤란로

**목표**
귀사의 발전과 함께 성장하며, 제 역량을 최대한 발휘하여 귀사의 주요 구성원, 즉 '꽃'이 되고자 합니다. 

**학력**
- **서울대학교**  
  경영학 학사  
  2018년 3월 - 2022년 2월

**경력**
- **ABC 마케팅 회사**  
  마케팅 인턴  
  2021년 6
**************************************************
